In [51]:
import os
import pandas as pd
import random

In [52]:
scientific_names = pd.read_excel("../datasets/birdsongs-combined/metadata/final_metadata.xlsx", sheet_name="SCIENTIFIC_NAMES", usecols=[0, 3])

In [53]:
scientific_names

,folder code,genus
0,abethr1,Turdus
1,abhori1,Oriolus
2,abythr1,Turdus
3,acafly,Empidonax
4,acowoo,Melanerpes
...,...,...
987,yetgre1,Atimastillas
988,yetvir,Vireo
989,yewgre1,Eurillas
990,zebdov,Geopelia


In [73]:
base_dict = "../datasets/birdsongs-combined/train_audio/"

genus_dict = {}
for genus in scientific_names["genus"].unique():
    folder_dict = {}
    folder_keys = [f for f in scientific_names[scientific_names["genus"] == genus]["folder code"].values]
    for key in folder_keys:
        folder_dict[key] = len(os.listdir(base_dict + key + "/"))
    
    folder_dict = dict(sorted(folder_dict.items(), key = lambda x: x[1], reverse=True))

    genus_dict[genus] = list(folder_dict.keys())

genus_folders = pd.Series(genus_dict)
genus_folders

Turdus          [amerob, clcrob, grethr1, rubrob, blbthr1, afr...
Oriolus                      [blhori1, abhori1, ingori1, bltori1]
Empidonax       [aldfly, wilfly, pasfly, leafly, acafly, dusfl...
Melanerpes              [acowoo, rebwoo, gilwoo, lewwoo, hofwoo1]
Elminia                                                 [afbfly1]
                                      ...                        
Buccanodon                                              [yesbar1]
Gymnoris                                                [yespet1]
Atimastillas                                            [yetgre1]
Eurillas                                                [yewgre1]
Geopelia                                                 [zebdov]
Length: 531, dtype: object

In [74]:
# task:
# split list into "groups" based on value count of genus occurrence (see genus_counts)
# create empty list "order"
# for X in range [1, 19]:
#   - shuffle list of species = 1, append to "order" 
#   - shuffle list of species > 1, append to order
#   - subtract 1 from all instances in genus_counts
#   - for all instances in genus_counts:
#       - if value <= 0, remove.

# final result:
# no species should occur for the Xth time until all instances of species with count >= X-1 have been displayed

In [75]:
# list of folder names to load IN THIS ORDER when training a model.
# this will be saved to a file for future use on other systems (thanks random)
final_order = []

max_folders_per_genus = 19 # this should not change???

for folder_count in range(1, max_folders_per_genus + 1):
    folders_to_add = []
    
    for genus, folders in genus_folders.items():
        if len(folders) >= 1:
            folders_to_add.append(folders[0])

            if len(folders) == 1:
                genus_folders[genus] = []
            elif len(folders) > 1:
                genus_folders[genus] = folders[1:]

    random.shuffle(folders_to_add)

    final_order += folders_to_add

992

In [85]:
test_dict = []

for folder in final_order:
    for genus in genus_dict.keys():
        if folder in genus_dict[genus]:
            test_dict.append([folder, genus])

# works perfectly.
test_frame = pd.DataFrame(test_dict[:532], columns=["folder", "genus"])

test_frame["genus"].value_counts()

genus
Cinnyris         2
Cecropis         1
Schistolais      1
Chlorospingus    1
Hirundo          1
                ..
Copsychus        1
Anous            1
Hydroprogne      1
Mimus            1
Antigone         1
Name: count, Length: 531, dtype: int64

In [92]:
# save final order to file.

with open("folder_order.txt", "w+") as f:
    f.writelines("\n".join(final_order))
    f.close()